# Install Dependencies

## External Dependencies

In [1]:
!pip install yfinance pandas

## FrostAura Dependencies

In [2]:
import os
import sys

USE_LOCAL_CODE: bool = True # NOTE: When changing this configuration, be sure to restart the interpreter in order to have sys path updates reflect.
directories_to_move_up: int = 2
current_executing_path: str = os.getcwd()
current_executing_path = current_executing_path.replace('\\', '/')
root_path: str = '/'.join(current_executing_path.split('/')[:-directories_to_move_up])

# Clean up.
%pip uninstall frostaura -y
sys.path = [p for p in sys.path if p != root_path]

if USE_LOCAL_CODE:
    sys.path.append(root_path)
else:
    %pip install -U --no-cache-dir frostaura

Note: you may need to restart the kernel to use updated packages.


## Logging

In [3]:
import logging

logging.basicConfig(level=logging.DEBUG)

# Import Dependencies

In [4]:
from frostaura.data_access import IPublicAssetDataAccess
from frostaura.data_access import YahooFinanceDataAccess
import pandas as pd

# Usage

In [5]:
instance: IPublicAssetDataAccess = YahooFinanceDataAccess()
symbol: str = 'AAPL'
request_url: str = f'https://finviz.com/quote.ashx?t={symbol}'
data: pd.DataFrame = instance.get_symbol_history(symbol=symbol)

data.head()

INFO:root:Fetching historical price movements for symbol "AAPL".
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): query2.finance.yahoo.com:443
DEBUG:urllib3.connectionpool:https://query2.finance.yahoo.com:443 "GET /v8/finance/chart/AAPL?period1=-631159200&period2=1660248742&interval=1d&includePrePost=False&events=div%2Csplits HTTP/1.1" 200 None


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1980-12-12,0.100039,0.100474,0.100039,0.100039,469033600,0.0,0.0
1980-12-15,0.095255,0.095255,0.094820,0.094820,175884800,0.0,0.0
1980-12-16,0.088296,0.088296,0.087861,0.087861,105728000,0.0,0.0
1980-12-17,0.090035,0.090470,0.090035,0.090035,86441600,0.0,0.0
1980-12-18,0.092646,0.093081,0.092646,0.092646,73449600,0.0,0.0


# Draft

In [6]:
from frostaura.data_access import IResourcesDataAccess
from frostaura.data_access import EmbeddedResourcesDataAccess
from frostaura.data_access import IPersonalAssetDataAccess
from frostaura.data_access import EasyEquitiesPersonalAssetDataAccess
from frostaura.models import ProfitCalculationResult

In [7]:
resource_data_access: IResourcesDataAccess = EmbeddedResourcesDataAccess()
personal_asset_data_access: IPersonalAssetDataAccess = EasyEquitiesPersonalAssetDataAccess(resource_data_access=resource_data_access,
                                                                         username=None,
                                                                         password=None)
holdings: list = personal_asset_data_access.get_personal_transactions()

In [8]:
from datetime import datetime

def interpolate_holdings_profits(holdings: dict) -> dict:
    '''Determine individual asset profit ratio & profit USD and interpolate them into a copy of the given holdings.'''

    __holdings__: dict = holdings.copy()
    
    for holding_symbol in __holdings__:
        context: dict = __holdings__[holding_symbol]
        history: pd.DataFrame = instance.get_symbol_history(symbol=context['symbol'])
        transactions_by_date_asc: list = sorted(context['transactions'], key=lambda i: i['date'])
        total_purchased_usd: float = 0
        total_purchased_shares: float = 0
        
        for transaction in transactions_by_date_asc:
            transaction_date: datetime = transaction['date']
            transaction_value: float = transaction['value']
            transaction_close: float = history.loc[transaction_date].Close

            transaction['usd'] = transaction_close * transaction_value
            total_purchased_usd += transaction_close * transaction_value
            total_purchased_shares += transaction_value

        total_current_usd: float = history.iloc[-1].Close * total_purchased_shares
        context['total_purchased_usd'] = total_purchased_usd
        context['total_purchased_shares'] = total_purchased_shares
        context['total_current_usd'] = total_current_usd
        context['total_profit_ratio'] = (1 - min(total_purchased_usd, total_current_usd) / max(total_purchased_usd, total_current_usd)) * 100
        context['total_profit_usd'] = total_current_usd - total_purchased_usd

    return __holdings__

In [9]:
def calculate_holdings_profit(holdings: dict) -> ProfitCalculationResult:
    '''Determine the holdings profit percentage & profit USD, given the holdings.'''

    __holdings_with_profits__: dict = interpolate_holdings_profits(holdings=holdings)
    total_purchased_usd: float = sum([__holdings_with_profits__[k]['total_purchased_usd'] for k in __holdings_with_profits__])
    total_current_usd: float = sum([__holdings_with_profits__[k]['total_current_usd'] for k in __holdings_with_profits__])
    total_profit_ratio: float = (1 - min(total_purchased_usd, total_current_usd) / max(total_purchased_usd, total_current_usd)) * 100
    total_profit_usd: float = total_current_usd - total_purchased_usd

    return ProfitCalculationResult(percentage=total_profit_ratio, value=total_profit_usd)

In [11]:
vars(calculate_holdings_profit(holdings=holdings))

INFO:root:Fetching historical price movements for symbol "TSLA".
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): query2.finance.yahoo.com:443
DEBUG:urllib3.connectionpool:https://query2.finance.yahoo.com:443 "GET /v8/finance/chart/TSLA?period1=-631159200&period2=1660248754&interval=1d&includePrePost=False&events=div%2Csplits HTTP/1.1" 200 None
INFO:root:Fetching historical price movements for symbol "AAPL".
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): query2.finance.yahoo.com:443
DEBUG:urllib3.connectionpool:https://query2.finance.yahoo.com:443 "GET /v8/finance/chart/AAPL?period1=-631159200&period2=1660248755&interval=1d&includePrePost=False&events=div%2Csplits HTTP/1.1" 200 None
INFO:root:Fetching historical price movements for symbol "DDD".
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): query2.finance.yahoo.com:443
DEBUG:urllib3.connectionpool:https://query2.finance.yahoo.com:443 "GET /v8/finance/chart/DDD?period1=-631159200&period2

{'percentage': 4.0128600078093495, 'value': 10.09648056573866}